In [29]:
# Reweighting a probability distribution to a different temperature
import numpy as np
def reweight_distribution(original_distribution, temperature=0.5) :
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution/np.sum(distribution)

import keras
import numpy as np

path = keras.utils.get_file('nietzsche.txt',
                           origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()


In [33]:
print('Corpus length:', len(text))
maxlen = 60
step=3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))
print('First sequences:', sentences[0])
print('First sequences follow by:', next_chars[0])


Corpus length: 600901
Number of sequences: 200281
First sequences: preface


supposing that truth is a woman--what then? is the
First sequences follow by: r


In [34]:

chars = sorted(list(set(text)))

print('Uniq Char:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences),maxlen,len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Uniq Char: 59
Vectorization...


In [39]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               96256     
_________________________________________________________________
dense_2 (Dense)              (None, 59)                7611      
Total params: 103,867
Trainable params: 103,867
Non-trainable params: 0
_________________________________________________________________


In [48]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1,preds, 1)
    return np.argmax(probas)

import random
import sys

for epoch in range(1, 5):
    print ('epoch', epoch)
    model.fit(x,y,batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) -maxlen -1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('\n------ temerature:', temperature)
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

epoch 1
Epoch 1/1
200281/200281 [==============================] - 101s 505us/step - loss: 1.6448
--- Generating with seed: "spirituality. of what use is it for
nimble, everyday intelle"

------ temerature: 0.2
spirituality. of what use is it for
nimble, everyday intellect, and the superinger and there is the sense of the sense of the sense of the self the sense of the sulf and the sense of the senses of the sense of the sense of the sense of the sure and the sense of the served to the are the persons of the superingered and there is the sense of the sense of the sense of the sense of the soul of the sense of the superinges and the most man and there is the sense
------ temerature: 0.5
se of the superinges and the most man and there is the sense of manifess is all all in a the will superiority to a comparation of the all the deperish, his instandus a man a understand to the manifest is one in the same and only regards of the
sance of the defering that is the "constince to the wears and

In [47]:
start_index = random.randint(0, len(text) -maxlen -1)
generated_text = text[start_index: start_index + maxlen]
print('--- Generating with seed: "' + generated_text + '"')
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('\n------ temerature:', temperature)
    sys.stdout.write(generated_text)
    for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        generated_text += next_char
        generated_text = generated_text[1:]
        sys.stdout.write(next_char)

--- Generating with seed: "t has recognized this
faculty as its sixth sense. owing to t"

------ temerature: 0.2
t has recognized this
faculty as its sixth sense. owing to the spirit and in the self--in the self-and a moral and the sense the self-grain and the sumple and a moral and a moral and the spirit and a stratition of the are a moral and a concrust and the spirit and the spirit and the spirit and the self--the spirit and the sensition of a moral and a morals and the are a moral and suppressition and the sense the his conspition and its is the sective and a mor
------ temerature: 0.5
he sense the his conspition and its is the sective and a moral, the souls and the sersed torder of the some that
the such a proves the by friend works of he a chist to has externation of the deperhal and to the sumple a the agring of the discivery in their actime of in the a dout the serves and muss and the intome in the prover to
discouls is the prodeders the love believe to the spirition of the fac